In [1]:
import tensorflow as tf

In [2]:
print(tf.__version__)

2.3.1


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import os

In [4]:
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

In [5]:
DIRECTORY = r"C:\Users\user\Desktop\Gender Detection\dataset"
print(os.listdir(DIRECTORY))

['man', 'woman']


In [6]:
CATEGORIES = ['man', 'woman']

In [7]:
data = []
labels = []

In [8]:
for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = load_img(img_path, target_size = (224, 224))
        image = img_to_array(image)
        image = preprocess_input(image)
        
        data.append(image)
        labels.append(category)

In [9]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [10]:
data = np.array(data, dtype="float32")
labels = np.array(labels)

In [12]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.20, stratify=labels, random_state=42)

In [13]:
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")


In [15]:
baseModel = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

In [16]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)


In [17]:
model = Model(inputs=baseModel.input, outputs=headModel)


In [18]:
for layer in baseModel.layers:
	layer.trainable = False

In [19]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])


In [20]:
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

Epoch 1/20
57/57 [==============================] - 81s 1s/step - loss: 0.6479 - accuracy: 0.6784 - val_loss: 0.4180 - val_accuracy: 0.8139
Epoch 2/20
57/57 [==============================] - 78s 1s/step - loss: 0.4029 - accuracy: 0.8103 - val_loss: 0.3250 - val_accuracy: 0.8701
Epoch 3/20
57/57 [==============================] - 78s 1s/step - loss: 0.3911 - accuracy: 0.8191 - val_loss: 0.3217 - val_accuracy: 0.8701
Epoch 4/20
57/57 [==============================] - 78s 1s/step - loss: 0.3275 - accuracy: 0.8582 - val_loss: 0.2968 - val_accuracy: 0.8810
Epoch 5/20
57/57 [==============================] - 78s 1s/step - loss: 0.3023 - accuracy: 0.8660 - val_loss: 0.2768 - val_accuracy: 0.8939
Epoch 6/20
57/57 [==============================] - 79s 1s/step - loss: 0.3051 - accuracy: 0.8671 - val_loss: 0.2800 - val_accuracy: 0.8918
Epoch 7/20
57/57 [==============================] - 79s 1s/step - loss: 0.2851 - accuracy: 0.8665 - val_loss: 0.2750 - val_accuracy: 0.8983
Epoch 8/20
57/57 [==

In [24]:
import cv2

In [26]:
image = cv2.imread("pic.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (224, 224))
image = np.array([image])
prediction = model.predict(image)
predIdxs = np.argmax(prediction, axis=1)
print(predIdxs)

[0]


In [27]:
predIdxs[0]

0

In [28]:
print(CATEGORIES[predIdxs[0]])

man
